In [ ]:
# !pip install bs4
# !pip install requests
# !pip install urllib
# !pip install selenium
!pip install HTMLParser

In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
# get a list of url-matches
# resp = requests.get(head_url)
chrome_opt = Options()
chrome_opt.headless = True
driver = webdriver.Chrome('driver/chromedriver',chrome_options=chrome_opt)

In [ ]:
driver.get(head_url)

In [ ]:
urls = driver.find_elements_by_class_name("nav-switch__next")

In [ ]:
ref = urls[0].get_attribute('data-href')
date = ref.split('date=')[1].split('%')[0]

In [ ]:
new_url = 'https://www.goal.com/en-us/premier-league/fixtures-results/' + date + '/2kwbbcootiqqgmrzs6o5inle5'
new_url

In [ ]:
driver.quit()

In [ ]:
# max_iter = 100
curr_url = 'https://www.goal.com/en-us/premier-league/fixtures-results/2021-01-02/2kwbbcootiqqgmrzs6o5inle5'
matches_url = list()
matches_url.append(curr_url)
driver.get(curr_url)
nxt_page = driver.find_elements_by_class_name("nav-switch__next")[0]
while nxt_page is not None:
    date = nxt_page.get_attribute('data-href').split('date=')[1].split('%')[0]
    nxt_url = 'https://www.goal.com/en-us/premier-league/fixtures-results/' + date + '/2kwbbcootiqqgmrzs6o5inle5'
    matches_url.append(nxt_url)
    driver.get(nxt_url)
    nxt_page = driver.find_elements_by_class_name("nav-switch__next")[0]

In [ ]:
"""
    Get url for match page
"""
match_prefix = 'https://www.goal.com'
match_pages = list()
for url in matches_url:
    driver.get(url)
    matches = driver.find_elements_by_class_name('match-main-data-link')
    for match in matches:
        href = match.get_attribute('href')
        splits = href.split('/')
        commentary_url = splits[0]+'//'+splits[1]+'/'+splits[2]+'/'+\
splits[3]+'/'+splits[4]+'/'+splits[5]+'/commentary-result/'+splits[6] 
        
        match_pages.append(commentary_url)

In [ ]:
with open('match_pages.txt', 'w') as f:
    for item in match_pages : f.write('%s\n' % item)

In [3]:
match_pages = None
with open('match_pages.txt', 'r') as f:
    match_pages = f.read().splitlines()
    print(len(match_pages))

3874


In [ ]:
"""
    Access match page and store event data and commentary data
"""
from tqdm.notebook import tqdm
driver.quit()
chrome_opt = Options()
chrome_opt.headless = True
import pandas as pd
df = pd.DataFrame(columns=['time', 'team_venue', 'event_type', 'score', 'main_player', 'support_text', 'commentary'])
cnt = 0

lst_time = list()
lst_venue = list()
lst_event_type = list()
lst_score = list()
lst_main_player = list()
lst_support_text = list()
lst_commentary = list()
driver = webdriver.Chrome('driver/chromedriver',options=chrome_opt)
# driver.implicitly_wait(3)

for url in tqdm(match_pages):
#     url = 'https://www.goal.com/en-us/match/tottenham-hotspur-v-leeds-united/commentary-result/8ka4lbdcciz7f38za3dwvjytw'
    driver.get(url)
    contents = driver.find_elements_by_xpath('/html/body/div[4]/div[2]/div[2]/div/div/div[1]/div[2]/div[1]/div[1]')
    for content in contents:
        for c in content.find_elements_by_class_name('event'):
#             if len(c) == 1 : continue
            team = None
            try: 
                team = c.find_elements_by_class_name('team-home')[0]
                lst_venue.append('home')
            except:
                try :
                    team = c.find_elements_by_class_name('team-away')[0]
                    lst_venue.append('away')
                except : continue
            # time
            lst_time.append(team.find_elements_by_class_name('event-time')[0].text)
            # event type
            event_type = c.find_elements_by_class_name('event-icon')[0].find_elements_by_tag_name('div')
            if len(event_type) == 2 :
                score = event_type[0].text
                lst_score.append(score)
                lst_event_type.append('goal')
            else : 
                lst_event_type.append(event_type[0].get_attribute('class').split('-')[-1])
                lst_score.append('na')
            # main_player
            lst_main_player.append(team.find_elements_by_class_name('event-text-main')[0].text)
            # support text
            lst_support_text.append(team.find_elements_by_class_name('event-text-additional')[0].text)
    if(cnt%100 == 0):
        driver.quit()
        driver = webdriver.Chrome('driver/chromedriver',options=chrome_opt)
        driver.implicitly_wait(3)

In [12]:
driver.quit()
chrome_opt = Options()
chrome_opt.headless = True
driver = webdriver.Chrome('driver/chromedriver',chrome_options=chrome_opt)
url = 'https://www.goal.com/en-us/match/wolverhampton-wanderers-v-west-bromwich-albion/commentary-result/8m580387s14kc4zbi0aug5fkk'
driver.get(url)

comment = driver.find_elements_by_class_name("comment")
# t = driver.find_elements_by_xpath("//")
print(len(comment))
for c in comment:
    # events
    try : 
        event_comment = c.find_element_by_class_name('comment-event')
        additional_info = event_comment.find_element_by_class_name('event-text-main').text
        info = c.find_element_by_class_name('comment-desc').find_element_by_class_name('text')
        print(event_comment.find_element_by_class_name('event-icon').find_element_by_tag_name('div').get_attribute('class').split('-')[-1])
        print(info.text)
        print(additional_info)
        #         time = info.find_elements_by_tag_name("span")
#         for t in time : print(t.text + ' yes')
    except : 
#         print('here')
        continue

/home/ROHITGANGURDE/anaconda3/envs/a7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


77
yellow_card


substitution


substitution


yellow_card


substitution


substitution


substitution


penalty_goal


assist


goal


goal


assist


goal


yellow_card


penalty_goal




In [16]:
url = 'https://www.goal.com/en-us/match/leicester-city-v-southampton/8lqo6rkgj6gmsitszdhwanf2s'
driver.get(url)

clicked = driver.find_element_by_css_selector("div.content-collapser.content-collapser--btn").click()
comment = driver.find_elements_by_class_name("comment")
# t = driver.find_elements_by_xpath("//")
print(len(comment))
for c in comment:
    # events
    try : 
        event_comment = c.find_element_by_class_name('comment-event')
        additional_info = event_comment.find_element_by_class_name('event-text-main').text
        info = c.find_element_by_class_name('comment-desc').find_element_by_class_name('text')
        print(event_comment.find_element_by_class_name('event-icon').find_element_by_tag_name('div').get_attribute('class').split('-')[-1])
        print(info.text)
        print(additional_info)
        #         time = info.find_elements_by_tag_name("span")
#         for t in time : print(t.text + ' yes')
    except : 
#         print('here')
        continue

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.content-collapser.content-collapser--btn"}
  (Session info: headless chrome=88.0.4324.182)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.18.0-240.15.1.el8_3.x86_64 x86_64)


In [ ]:
contents = driver.find_elements_by_xpath('/html/body/div[4]/div[2]/div[2]/div/div/div[1]/div[2]/div[1]/div[1]')
for content in contents:
    for c in content.find_elements_by_class_name('event'):
        team = c.find_elements_by_class_name('team-home')[0]
        ev_icon = c.find_elements_by_class_name('event-icon')[0]
        if len(ev) == 2 :
            score = ev[0].find_elements_by_class_name('match-event-score')[0].text
            print(score)
        else : 
            ev = ev_icon.find_elements_by_tag_name('div')
            print(len(ev))
            print(ev[0].get_attribute('class').split('-')[-1])
        print()
#             lst_score.append('na')
#         try: 
#             team = c.find_elements_by_class_name('event-text team-home clearfix')
#             lst_venue.append('home')
#         except:
#             team = c.find_elements_by_class_name('event-text team-away clearfix')
#             lst_venue.append('away')

In [ ]:
/html/body/div[4]/div[2]/div[2]/div/div/div[1]/div[2]/div[1]/div[1]/div[5]

In [ ]:
driver.quit()

In [ ]:
driver.get(matches_url[0])

In [91]:
url = 'https://www.goal.com/en-us/match/brighton-hove-albion-v-wolverhampton-wanderers/8jhrtgfo31i2az79l5e6d2adw'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [34]:
event = soup.find_all('div', class_='comment-event')[0]
desc = soup.find_all('div', class_='comment-desc')[0]

In [35]:
event.text

'       Fábio Silva   Yellow Card       '

In [36]:
desc.text

'  It is a midweek trip to West Ham next up for the Baggies in the Premier League - but Wolves are back in FA Cup action, with a fourth round tie against Chorley. Until then however, thank you for joining us - and have a great weekend!  '

In [37]:
soup.select(".comment")[10].get_text()

"  84'  Gibbs has a clumsy nibble at Traore and concedes the free-kick around 35 yards out in midfield. The delivery is dangerous but Button manages to make a safe connection to collect.   "

In [38]:
soup.select(".comment .comment-event")[0].get_text()

'       Fábio Silva   Yellow Card       '

In [98]:
for page in match_pages:
    splits = page.split('/')
    url = splits[0] + '//'
    for s in splits[3:] : url += s + '/'
    

https://www.goal.com/en-us/match/tottenham-hotspur-v-leeds-united/commentary-result/8ka4lbdcciz7f38za3dwvjytw/
https://www.goal.com/en-us/match/crystal-palace-v-sheffield-united/commentary-result/8jrh8s08xe1ih3qztdvfdm710/
https://www.goal.com/en-us/match/brighton-hove-albion-v-wolverhampton-wanderers/commentary-result/8jhrtgfo31i2az79l5e6d2adw/
https://www.goal.com/en-us/match/west-bromwich-albion-v-arsenal/commentary-result/8kdba6nqqw2at21dxne4t0hlg/
https://www.goal.com/en-us/match/newcastle-united-v-leicester-city/commentary-result/8k3h8bepmyl3x0i91zk7n6rys/
https://www.goal.com/en-us/match/chelsea-v-manchester-city/commentary-result/8jobnl84s86y0zxac38tbgw7o/
https://www.goal.com/en-us/match/southampton-v-liverpool/commentary-result/8k6wtn5hn7whnxgbfvmkeuy38/
https://www.goal.com/en-us/match/sheffield-united-v-newcastle-united/commentary-result/8kyga41m1kxtb32eqzv5z5fro/
https://www.goal.com/en-us/match/burnley-v-manchester-united/commentary-result/85l57jh7jxgumy8faimtcxe6s/
https

In [99]:
from tqdm.notebook import tqdm

terms = ['Yellow Card', 'Penalty Goal', 'Assist', 'Goal', 'Red Card']
lst_time = list()
lst_player_sub = list()
lst_event_type = list()
# lst_score = list()
lst_main_player = list()
# lst_support_text = list()
lst_commentary = list()
for page in tqdm(match_pages) :
    splits = page.split('/')
    url = splits[0] + '//'
    for s in splits[3:] : url += s + '/'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    event = soup.find_all('div', class_='comment')
    for e in event:
        try :
            ev = e.find('div', class_='comment-event').text.split()
            lst_main_player.append(ev[0] + ' ' + ev[1])
            rest = ''
            for r in ev[2:]: rest += r + ' '
            rest = rest.strip()
            if rest in terms : lst_event_type.append(rest)
            else :
                lst_event_type.append('Substitution')
                lst_player_sub.append(rest)
            info = e.find('div', class_='comment-desc').text.split()
            time = info[0]
            start = 1
            if info[1] == '+' : 
                time += ' + ' + info[2]
                start = 3
            lst_time.append(time)
            commentary = ''
            for i in info[start:] : commentary += i + ' '
            commentary = commentary.strip()
            lst_commentary.append(commentary)
    #         print("Main Player : {0}\nEvent : {1}\nTime : {2}\nCommentary:{3}".format(main_player, event_type,
    #                                                                                   time, commentary))
    #         print('\n\n')
        except: 
    #         print('her')
            continue

  0%|          | 0/3874 [00:00<?, ?it/s]

In [109]:
len(lst_player_sub)

20031

In [110]:
lst_player_sub = list()
for page in tqdm(match_pages) :
    splits = page.split('/')
    url = splits[0] + '//'
    for s in splits[3:] : url += s + '/'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    event = soup.find_all('div', class_='comment')
    for e in event:
        try :
            ev = e.find('div', class_='comment-event').text.split()
#             lst_main_player.append(ev[0] + ' ' + ev[1])
            rest = ''
            for r in ev[2:]: rest += r + ' '
            rest = rest.strip()
            if rest not in terms : lst_player_sub.append(rest)
            else : lst_player_sub.append('')
        except: continue

  0%|          | 0/3874 [00:00<?, ?it/s]

In [111]:
import pandas as pd

df = pd.DataFrame({'time' : lst_time,
                   'event' : lst_event_type,
                   'player_sub' : lst_player_sub,
                   'main_player' : lst_main_player,
                   'commentary' : lst_commentary})
df.head()

,time,event,player_sub,main_player,commentary
0,89',Yellow Card,,K. Phillips,A hasty challenge from Kalvin Phillips now and...
1,87',Substitution,H. Kane,Carlos Vinícius,Harry Kane - who became a father this week - m...
2,85',Yellow Card,,P. Højbjerg,"Hojbjerg picks up a late booking here, as the ..."
3,78',Substitution,T. Ndombèlé,Lucas Moura,"Tottenham make their second change now, with N..."
4,76',Substitution,H. Winks,M. Sissoko,Lloris gets a glance to this one and Ayling's ...


In [112]:
df.to_pickle('event_data.pickle')